<div style="padding:30px; color: white; background-color: #0071CD">
<center>
    <img src="img/logoub.jpeg"></img>
<p>
<h1>Algorítmica Avanzada</h1>
<h2>Problemas Enumerativos</h2>
</center>
</p>
</div>

<div class="alert alert-info">
<center>
  <h1>Backtracking</h1>
</center>
</div>

<div class="alert alert-success" style="width:90%; margin:0 auto;">
<h2><p>0. Cartas</p></h2>
<p>
Supongamos que tenemos un conjunto de parejas de cartas del mismo valor. En particular, tenemos parejas de cartas hasta N. Por ejemplo, para $N=3$ tendríamos una pareja de 1s, una de 2s y una de 3s (un total de 6 cartas). Lo que queremos es encontrar la manera de ordenarlas para que entre las dos cartas de valor $n_i$ haya exactamente $n_i$ cartas. En el ejemplo con $N=3$ tendríamos la siguiente solución:
</p>
<center> <b>   3 - 1 - 2 - 1- 3 - 2 </b></center>
<p>
Podemos observar que entre los 3 hay tres cartas, entre los 2 hay dos y entre los 1 solo hay una. Dependiendo del tamaño de N el problema podría no tener solución. Por ejemplo $N=5$ o $N=6$. Se pide la implementación de un algoritmo que, mediante backtracking, devuelva una lista con la configuración encontrada dado un valor N. En caso de no haber solución, debe devolver una lista vacía.
</p> <p>
Es importante destacar que tendréis que prestar especial atención a la complejidad de vuestra solución, dado que el problema crece de forma no polinómica y para un valor $N>12$ podría tardar horas. Hay diferentes aproximaciones para resolverlo. Bien hecho, $N=12$ debería tardar escasos segundos. Si haces varias versiones, puedes entregarlas junto con la comparativa de complejidad correspondiente.</p>
</div>

In [4]:
'''
Implementa aquí tu solución. Debe devolver una lista con las cartas ordenadas tal y como se especifica
en el enunciado o None en caso de no existir solución para la entrada dada.
'''
def solve_deck(n, comb):
    
    if n==1:
        index = comb.index(0)
        if index < len(comb)-2:
            if comb[index+2]==0:
                comb[index] = 1
                comb[index+2] = 1
                return True
        
        return False
    
    #look for the first possition not used
    index = comb.index(0)
    #proced if we can place n in the possition of index and in index+n+1, cause gotta be n cards between them
    while index < len(comb)-n-1:
        #if the possition where the second card is put n in both possitions
        if comb[index+n+1]==0:
            comb[index] = n
            comb[index+n+1] = n
            
            #check if with this combination we can reach a solution
            check = solve_deck(n-1, comb)
            if check:
                return True
            
            #if not return the values to zero
            comb[index] = 0
            comb[index+n+1] = 0
        
        #as it failed compute the next possition not used if there's any
        index += 1
        if 0 in comb[index:]:
            index = comb.index(0,index)
        
    return False

def deck(n):
    
    if n==1:
        return []
    
    comb = [0 for x in range(2*n)]
    check = solve_deck(n, comb)
    
    if check:
        return comb
    
    return []


In [5]:
for n in range(1,13):
    sol = deck(n)
    if sol:
        print("Sol "+str(n)+": ","-".join(map(str,sol)))
    else:
        print("Sol "+str(n)+": NO ENCONTRADA")

Sol 1: NO ENCONTRADA
Sol 2: NO ENCONTRADA
Sol 3:  3-1-2-1-3-2
Sol 4:  4-1-3-1-2-4-3-2
Sol 5: NO ENCONTRADA
Sol 6: NO ENCONTRADA
Sol 7:  7-3-6-2-5-3-2-4-7-6-5-1-4-1
Sol 8:  8-3-7-2-6-3-2-4-5-8-7-6-4-1-5-1
Sol 9: NO ENCONTRADA
Sol 10: NO ENCONTRADA
Sol 11:  11-6-10-2-9-3-2-8-6-3-7-5-11-10-9-4-8-5-7-1-4-1
Sol 12:  12-10-11-6-4-5-9-7-8-4-6-5-10-12-11-7-9-8-3-1-2-1-3-2


## Analiza el coste de tu/s implementación/es:

#### Aquí tu respuesta

<div class="alert alert-info">
<center>
  <h1>Ramificación y poda</h1>
</center>
</div>


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>1. Mochila</p></h2>
  
   <p> En esta primera sección trabajaremos con el problema de la mochila que ya vimos en problemas Greedy y PD. Refrequemos un poco (por si aún no lo tenéis claro):
</p>
    Nos encontramos en una habitación en la que hay $N$ objetos con pesos $w_1, w_2, w_3 ... w_N$ y tenemos una mochila que puede soportar una carga máxima de $W$. En este caso se pide que realicéis una implementación de Ramificación y Poda para resolver el problema de la mochila con valor:
<br><br>
 **mochila_valor:** Cada objeto tendrá asignado un valor $v_1, v_2, v_3 ... v_N$. Buscamos llenar la mochila maximizando el valor total de la mochila pero sin superar la capacidad máxima. Es decir queremos encontrar la combinación de objetos $b$ tal que $\arg_{b} \max{\sum_{i=0}^{N}{v_i · b_i}}$ con la condición de que $\sum_{i=0}^{N}{(w_i · b_i)}\leq W$.
     <br><br>
Trabajaremos únicamente con la versión de la mochila en la que tenemos una cantidad limitada $c_1, c_2, c_3 ... c_N$ de objetos.</div>


In [3]:
import random as rd

# Genera un conjunto de objetos de prueba en forma de una lista de listas, 
# cada una de las cuales representa un elemento [peso,valor,cantidad]
def random_objects():
    objects = [[p,v,c] for p,v,c in zip(rd.sample(range(1, 50), k=10), rd.sample(range(1, 100), k=10),rd.sample(range(1, 15), k=10))]
    return objects

In [12]:
''' 
Implementa aquí la solución de Ramificación y poda que resuelve el problema de la mochila
@input: (D) Lista de listas con la forma [peso,valor,cantidad] representando los objetos que podemos escoger.
        (W) Entero que representa la capacidad de la mochila.
@output: Lista de listas con la forma [peso,valor,cantidad] representando los objetos escogidos.
'''
def knapsack_branch(D,W):
    D = sorted(D, key=lambda x: x[1]/x[0], reverse=True)
    
    max_profit = 0
    
    #Create an empty list that will work as a queue
    Q = []
    
    #Create a list with u[level, profit, bound, weight]
    #inicialize the first element to profit=weight=0
    u = [-1, 0, None, 0]
    
    Q.append(u)
    
    while Q:
        
        u = Q.pop()
        
        v = [None for x in range(4)]
        #if its the first element to get assign it level = 0
        if u[0]==-1:
            v[0] = 0
        
        
        
        
    
    
    return D

IndentationError: expected an indented block (<ipython-input-12-388b936e37a1>, line 31)

In [11]:
#rd.seed(0) # fijando el seed obtenemos siempre la misma mochila
D = random_objects()
print("D",D)
W = 150
print(knapsack_branch(D,W))

D [[29, 34, 14], [7, 40, 11], [36, 16, 6], [18, 76, 7], [25, 38, 4], [23, 52, 2], [8, 56, 5], [11, 45, 10], [10, 21, 3], [6, 19, 12]]
told cha
[[8, 56, 5], [7, 40, 11], [18, 76, 7], [11, 45, 10], [6, 19, 12], [23, 52, 2], [10, 21, 3], [25, 38, 4], [29, 34, 14], [36, 16, 6]]


### Analiza el coste de tu implementación. Comparalo además con el coste de las implementaciones greedy y dinámica. Compara también la optimalidad de la solución en los tres casos.

#### Aquí tu respuesta


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>2 - Orden de Ensamblaje</p></h2>
  
  <div class="row">
  <div>
  <p style="text-align: justify; padding-right: 30px">
La construcción de un artefacto requiere ensamblar $M$ piezas que identiﬁcamos con números entre $1$ y $M$. El coste de ensamblar la pieza $i$ depende del número de piezas ya ensambladas. Los costes, que son valores positivos, se nos proporcionan en una matriz $C$. La celda $c_{ij}$ es el coste de ensamblar la pieza $i$ cuando ya se han ensamblado $j$ piezas. Deseamos calcular el orden de ensamblado de menor coste total, entendido éste como la suma del coste de ensamblaje de cada una de las piezas. 
  Implementa mediante ramificación y poda una solución a este problema con tal de encontrar el listado ordenado de piezas cuyo coste total de ensamblaje sea mínimo.
  </p>
  </div>
    </div>


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Código </h3></p> </center> 

<p>
<h3>INPUT</h3>
<ul>
<li>C: Matriz de costes de tamaño $M\times M$, donde M es el número de piezas. El elemento $i,j$ de la matriz representa el coste de ensamblaje para la pieza $i$ tras haber colocado $j$ piezas. 
</li>
</ul>
<br>
<h3>OUTPUT</h3>
Lista de números de $1$ a $M$ ordenados de forma que el coste total de ensamblaje sea mínimo. Ej: [5, 3, 2, 4, 1]
</p>

</div>

In [ ]:
def branch_ensemble(C):
    """
    Función que obtiene el orden óptimo de las piezas tal que el coste total es mínimo.
    Args:
        - C: matriz de costes M x M
    """
    items=[]
    
    return items

In [ ]:
from random import uniform

M = 10 # Número de piezas
# Matriz de costes generada aleatoriamente
C = {}
for i in range(M):
    C[i] = {}
    for j in range(M):
        C[i][j] = uniform(1, 25)

solucion = branch_ensemble(C)
print(solucion)

### En esta sección debéis responder a las preguntas siguientes:
#### 1. Explica cómo has realizado tu implementación, enfatizando en la heurística empleada.
#### 2. Análisis de complejidad de la implementación

#### Escribe aquí tus respuestas